In [3]:
import haystack
from haystack import Document
from haystack.document_stores.in_memory import InMemoryDocumentStore
from haystack.components.retrievers.in_memory import InMemoryBM25Retriever
from haystack.components.builders import PromptBuilder
from haystack import Pipeline
from haystack_integrations.components.generators.ollama import OllamaGenerator
import os
import json
import gradio as gr
import pandas as pd

dataset = pd.read_csv(r"C:\Users\anjan\OneDrive\Desktop\project\myenv\share\man\anime-dataset-2023.csv")

docs = [
    Document(
        content=(
            f"Synopsis: {row['Synopsis']}\n"
            f"Name: {row['Name']}\n"
            f"Genres: {row['Genres']}\n"
            f"Episodes: {row['Episodes']}\n"
            f"Rating: {row['Rating']}\n"
            f"Aired: {row['Aired']}\n"
            f"Premiered: {row['Premiered']}\n"
            f"Rank: {row['Rank']}\n"
            f"Studios: {row['Studios']}\n"
            f"Type: {row['Type']}\n"
            f"Status: {row['Status']}\n"
            f"Source: {row['Source']}\n"
            f"Duration: {row['Duration']}\n"
            f"Image URL: {row['Image URL']}\n"
            f"English name: {row['English name']}\n"
            f"Other name: {row['Other name']}\n"
            f"Score: {row['Score']}\n"
        ),
        meta={
            "Name": row["Name"],
            "Genres": row["Genres"],
            "Episodes": row["Episodes"],
            "Rating": row["Rating"],
            "Studios": row["Studios"],
            "Type": row["Type"],
            "Status": row["Status"],
            "Source": row["Source"],
            "Duration": row["Duration"],
            "Image URL": row["Image URL"],
            "anime_id": row["anime_id"],
            "English name": row["English name"],
            "Other name": row["Other name"],
            "Score": row["Score"],
        }
    )
    for _, row in dataset.iterrows()
]


document_store = InMemoryDocumentStore()
document_store.write_documents(docs)
retriever = InMemoryBM25Retriever(document_store=document_store)

template ="""
You are a helpful assistant. You will be given a question and you have to answer it based on the context provided. If the context does not contain enough information to answer the question, say "I don't know".
Context: 
{% for doc in documents %}
{{ doc.content }}
{% endfor %}
Question: {{question}}
Answer:
"""
prompt_builder = PromptBuilder(template=template,required_variables=["documents", "question"])

generator =OllamaGenerator(
    model = 'tinyllama',
    url = 'http://localhost:11434',
    generation_kwargs={
        "num_predict":100,
        "temperature":0.7,
    })


pipeline = Pipeline()
pipeline.add_component("retriever",retriever)
pipeline.add_component("prompt_builder", prompt_builder)
pipeline.add_component("llm", generator)
pipeline.connect("retriever", "prompt_builder.documents")
pipeline.connect("prompt_builder", "llm")

pipeline.draw("pipeline2.png")

def predict(question):
    result = pipeline.run({
        "retriever": {"query": question},
        "prompt_builder": {"question": question}
        })


    return result["llm"]["replies"][0]

gradio_interface = gr.Interface(
    fn=predict,
    inputs=gr.Textbox(label="Enter your question"),
    outputs=gr.Textbox(label="Answer"),
    title="Anime Q&A",
    description="Ask any question about the anime dataset and get an answer.",
    theme="default",
    allow_flagging="never",
)
gradio_interface.launch()





c:\Users\anjan\OneDrive\Desktop\project\myenv\Lib\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7865
* To create a public link, set `share=True` in `launch()`.
